# Практическое задание 4
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО: Смирнов Александр Львович

### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе. 

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils) 


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса. 
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой. 

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно 

## Предобработка данных (2 балла)

In [1]:
import sys
PATH_TO_CLAI_UTILS = "./utils/"
sys.path.append(PATH_TO_CLAI_UTILS)

In [2]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

Setting bashlex grammar using file: /home/furiousteabag/Projects/NLP/04_generation/./utils/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [3]:
train_data = pd.read_csv('data/train.csv')
train_data.head()

,invocation,cmd
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`


В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [4]:
test_data = pd.read_csv('data/test_data.csv')
test_data.head()

,invocation,cmd,origin
0,create ssh connection to specified ip from spe...,ssh user123@176.0.13.154,handcrafted
1,"search for commands containing string ""zeppeli...",history | grep zeppelin,handcrafted
2,search for location of specified file or appli...,whereis python3,handcrafted
3,grant all rights to root folder,sudo chmod 777 -R /,handcrafted
4,search in running processes for specified name,ps -aux | grep zepp,handcrafted


**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [5]:
import regex

from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
stemmer = PorterStemmer()
tokenizer = WordPunctTokenizer()

In [7]:
# looking up for specific stop words
tfidf = TfidfVectorizer(max_df=0.3, min_df=1)
tfidf.fit(train_data["invocation"].apply(lambda x: " ".join([stemmer.stem(t) for t in x.split()])))
specific_stop_words = tfidf.stop_words_
print(specific_stop_words)

{'and', 'the', 'in', 'current', 'all', 'file', 'directori'}


In [8]:
def clean_token(token):
    token = token.lower()
    token = regex.sub(r'[^\p{Latin}]', '', token)
    token = stemmer.stem(token)
    return token if token not in specific_stop_words else ""

In [9]:
def clean_text(text):
    return " ".join([clean_token(token) for token in tokenizer.tokenize(text) if clean_token(token) != ""])

In [10]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [11]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

In [12]:
train_data.head()

,invocation,cmd,text_cleaned,cmd_cleaned
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...,copi loadabl kernel modul mymodul ko to driver...,cp File $( uname -r )
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE,display line contain ipmrout kernel s compil t...,cat $( uname -r ) | grep Regex
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`,display run kernel s compil time config,cat $( uname -r )
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*,find loadabl modul for kernel whose name inclu...,find Path $( uname -r ) -regex Regex
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`,look for ani instanc of highmem kernel s compi...,grep Regex $( uname -r )


Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [13]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [14]:
import sentencepiece as spm
import io

In [15]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2
UNK_ID = 3


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 64

In [16]:
#! mkdir -p ./tokenizers

text_tokenizer = io.BytesIO()
spm.SentencePieceTrainer.Train(
    sentence_iterator=iter(train_data["text_cleaned"]),
    model_writer=text_tokenizer,
    #model_prefix='./tokenizers/sp_text',
    vocab_size=2400,
    pad_id=PAD_ID,                
    bos_id=BOS_ID,
    eos_id=EOS_ID,
    unk_id=UNK_ID
)
text_tokenizer = spm.SentencePieceProcessor(model_proto=text_tokenizer.getvalue())

cmd_tokenizer = io.BytesIO()
spm.SentencePieceTrainer.Train(
    sentence_iterator=iter(train_data["cmd_cleaned"]),
    model_writer=cmd_tokenizer,
    #model_prefix='./tokenizers/sp_cmd',
    vocab_size=500,
    pad_id=PAD_ID,                
    bos_id=BOS_ID,
    eos_id=EOS_ID,
    unk_id=UNK_ID
)
cmd_tokenizer = spm.SentencePieceProcessor(model_proto=cmd_tokenizer.getvalue())

In [17]:
text = train_data["text_cleaned"][0]
print("Initial text:")
print(text)
print("\n")

tokenized = text_tokenizer.tokenize(text, add_bos=True, add_eos=True)
print("Indexes:")
print(tokenized)
print("\n")
print("Decoded version:")
print(text_tokenizer.decode(tokenized))
print("\n")
print("Pieces:")
print(*text_tokenizer.IdToPiece(tokenized))

Initial text:
copi loadabl kernel modul mymodul ko to driver modul matchig kernel


Indexes:
[1, 96, 11, 1442, 158, 567, 447, 20, 509, 473, 200, 618, 473, 343, 79, 7, 820, 42, 509, 473, 61, 11, 135, 567, 447, 20, 2]


Decoded version:
copi loadabl kernel modul mymodul ko to driver modul matchig kernel


Pieces:
<s> ▁cop i ▁load abl ▁ker ne l ▁mod ul ▁my mod ul ▁k o ▁to ▁drive r ▁mod ul ▁match i g ▁ker ne l </s>


In [18]:
cmd = train_data["cmd_cleaned"][0]
print("Initial cmd:")
print(cmd)
print("\n")

tokenized = cmd_tokenizer.tokenize(cmd, add_bos=True, add_eos=True)
print("Indexes:")
print(tokenized)
print("\n")
print("Decoded version:")
print(cmd_tokenizer.decode(tokenized))
print("\n")
print("Pieces:")
print(*cmd_tokenizer.IdToPiece(tokenized))

Initial cmd:
cp File $( uname -r )


Indexes:
[1, 101, 4, 15, 4, 66, 41, 4, 18, 13, 5, 53, 4, 42, 2]


Decoded version:
cp File $( uname -r )


Pieces:
<s> ▁cp ▁ File ▁ $ ( ▁ u name ▁- r ▁ ) </s>


**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [19]:
class TextToBashDataset(Dataset):
    
    def __init__(self, texts, cmds, text_tokenizer, cmd_tokenizer,
                 max_text_length=MAX_TEXT_LENGTH, max_code_length=MAX_CODE_LENGTH):
        
        self.text_tokenizer = text_tokenizer
        self.cmd_tokenizer = cmd_tokenizer
        self.max_text_length = max_text_length
        self.max_code_length = max_code_length
        
        self.items = []
        
        for text, cmd in zip(texts, cmds):
            text_tokenized = text_tokenizer.tokenize(text, add_bos=True, add_eos=True)
            cmd_tokenized = cmd_tokenizer.tokenize(cmd, add_bos=True, add_eos=True)
            if len(text_tokenized) > max_text_length:
                text_tokenized = text_tokenized[:max_text_length-1] + text_tokenized[-1:]
            if len(cmd_tokenized) > max_code_length:
                cmd_tokenized = cmd_tokenized[:max_code_length-1] + cmd_tokenized[-1:]
            self.items.append((text_tokenized, cmd_tokenized))
            
    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

In [20]:
train_ds = TextToBashDataset(
    texts=train_data["text_cleaned"],
    cmds=train_data["cmd_cleaned"],
    text_tokenizer=text_tokenizer,
    cmd_tokenizer=cmd_tokenizer)

valid_ds = TextToBashDataset(
    texts=valid_data["text_cleaned"],
    cmds=valid_data["cmd_cleaned"],
    text_tokenizer=text_tokenizer,
    cmd_tokenizer=cmd_tokenizer)

In [21]:
text_idx, cmd_idx = valid_ds[0]
print(valid_data.iloc[0])
print("\n")
print(text_idx)
print(text_tokenizer.decode(text_idx))
print(cmd_tokenizer.decode(cmd_idx))

invocation      searches through the root filesystem ("/") for...
cmd                          find / -name Chapter1 -type f -print
text_cleaned    search through root filesystem for name chapte...
cmd_cleaned                  find Path -name Regex -type f -print
Name: 9843, dtype: object


[1, 18, 422, 125, 264, 312, 13, 15, 1051, 148, 192, 17, 254, 2]
search through root filesystem for name chapter print locat
find Path -name Regex -type f -print


In [22]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):

    text_idxs = [torch.tensor(item[0]) for item in batch]
    cmd_idxs = [torch.tensor(item[1]) for item in batch]
    
    text_idxs = pad_sequence(text_idxs, padding_value=PAD_ID, batch_first=True)
    cmd_idxs = pad_sequence(cmd_idxs, padding_value=PAD_ID, batch_first=True)
    
    return text_idxs, cmd_idxs 

In [23]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn),
}

In [24]:
next(iter(loaders["valid"]))

(tensor([[  1,  18, 422,  ...,   0,   0,   0],
         [  1,  18, 422,  ...,   0,   0,   0],
         [  1,  18, 422,  ...,   0,   0,   0],
         ...,
         [  1, 307,  23,  ...,   0,   0,   0],
         [  1, 307, 181,  ...,   0,   0,   0],
         [  1, 307, 181,  ...,   0,   0,   0]]),
 tensor([[ 1,  9, 14,  ...,  0,  0,  0],
         [ 1,  9, 14,  ...,  0,  0,  0],
         [ 1,  9, 14,  ...,  0,  0,  0],
         ...,
         [ 1,  4, 62,  ...,  0,  0,  0],
         [ 1, 64, 11,  ...,  0,  0,  0],
         [ 1, 64, 11,  ...,  0,  0,  0]]))

## Обучение бейзлайна (2 балла)

In [25]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [26]:
text_model_config = BertConfig(
    vocab_size = text_tokenizer.vocab_size(),
    hidden_size = 256,
    num_hidden_layers = 2,
    num_attention_heads = 8,
    intermediate_size = 256 * 4,
    hidden_dropout_prob = 0.1,
    pad_token_id = PAD_ID,
)

cmd_model_config = BertConfig(
    vocab_size = cmd_tokenizer.vocab_size(),
    hidden_size = 256,
    num_hidden_layers = 2,
    num_attention_heads = 8,
    intermediate_size = 256 * 4,
    hidden_dropout_prob = 0.1,
    pad_token_id = PAD_ID,
    is_decoder = True,
    add_cross_attention = True
)

print(cmd_model_config.is_decoder)
print(cmd_model_config.add_cross_attention)

True
True


In [27]:
config = EncoderDecoderConfig.from_encoder_decoder_configs(text_model_config, cmd_model_config)
model = EncoderDecoderModel(config=config)

In [28]:
text_idxs, cmd_idxs = next(iter(loaders["valid"]))
decoder_input = cmd_idxs[..., :-1]
target = cmd_idxs[..., 1:]

print(text_idxs.shape)
print(decoder_input.shape)
print(target.shape)

torch.Size([64, 26])
torch.Size([64, 39])
torch.Size([64, 39])


In [29]:
text_idxs_mask = torch.where(text_idxs != PAD_ID, 1, 0)
decoder_input_mask = torch.where(decoder_input != PAD_ID, 1, 0)

In [30]:
cmd_idxs[..., :-1]

tensor([[ 1,  9, 14,  ...,  0,  0,  0],
        [ 1,  9, 14,  ...,  0,  0,  0],
        [ 1,  9, 14,  ...,  0,  0,  0],
        ...,
        [ 1,  4, 62,  ...,  0,  0,  0],
        [ 1, 64, 11,  ...,  0,  0,  0],
        [ 1, 64, 11,  ...,  0,  0,  0]])

In [31]:
out = model(
    input_ids=text_idxs, decoder_input_ids=decoder_input,
    attention_mask=text_idxs_mask, decoder_attention_mask=decoder_input_mask)

logits = out.logits
print(logits.shape)

torch.Size([64, 39, 500])


### eval

In [32]:
device = torch.device('cpu')

In [33]:
def greedy_decode(text_tokenized, model, max_len=20):
    
    model.eval()
    model = model.to(device)
    text_tokenized = text_tokenized.unsqueeze(0).to(device)
    cmd_prediction = torch.tensor([BOS_ID]).unsqueeze(0).to(device)
    next_piece = BOS_ID
    
    while next_piece != EOS_ID and cmd_prediction.shape[-1] < max_len:
        out = model(input_ids=text_tokenized, decoder_input_ids=cmd_prediction)
        next_piece = torch.argmax(out.logits.squeeze(0)[-1]).item()
        cmd_prediction = torch.cat((cmd_prediction, torch.tensor([[next_piece]])), dim=1)
        
    return cmd_prediction[0]

In [34]:
def eval_decode(i, data, ds, model, max_len=20):
    
    for k, v in dict(data.iloc[i]).items():
        print(f"{k}: {v}")
    text_tokenized = torch.tensor(ds[i][0])
    cmd_prediction = greedy_decode(text_tokenized, model, max_len)
    cmd_prediction = [x.item() for x in cmd_prediction]
    print(f"cmd_predicted: {cmd_tokenizer.decode(cmd_prediction)}")

In [35]:
import random

def random_eval(data, ds, model, sample_size=5, max_len=20):
    indexes = random.sample(range(len(ds)), sample_size)
    for i in indexes:
        eval_decode(i, data, ds, model, max_len)
        print()

In [36]:
eval_decode(3, train_data, train_ds, model)

invocation: find all loadable modules for current kernel, whose name includes "perf"
cmd: find /lib/modules/`uname -r` -regex .*perf.*
text_cleaned: find loadabl modul for kernel whose name includ perf
cmd_cleaned: find Path $( uname -r ) -regex Regex
cmd_predicted: cptpt{ commsTYngfin wpan ciceyCheckingskipgeoutilepan


In [37]:
random_eval(train_data, train_ds, model)

invocation: gets ip address of 'en0' selected network interface.
cmd: ifconfig en0 | awk '/inet addr/ {gsub("addr:", "", $2); print $2}'
text_cleaned: get ip address of en select network interfac
cmd_cleaned: ifconfig Regex | awk Program
cmd_predicted: cptpt{ commskipiletor-1mountemp ciceyCheckingskipgeouticct

invocation: change all directories under "./storage/" to owner "apache" and group "laravel"
cmd: sudo find ./storage/ -type d -exec chown apache:laravel {} \;
text_cleaned: chang under storag to owner apach group laravel
cmd_cleaned: find Path -type d -exec chown Regex {} \;
cmd_predicted: cptpt{ commsTYngfin wpan ciceyCheckingskipgeoutilepan

invocation: calculate the md5 sum of all files in "/your/dir" including content and filenames
cmd: grep -ar -e . /your/dir | md5sum | cut -c-32
text_cleaned: calcul md sum of your dir includ content filenam
cmd_cleaned: grep -a -r -e Regex Regex | md5sum | cut -c -Number
cmd_predicted: cptpt{ commsTYngfinpruneout ciceyCheckingskipgeoutilep

In [38]:
random_eval(valid_data, valid_ds, model)

invocation: this find command substitute space with underscore in the file name replaces space in all the *.mp3 files with _
cmd: find . -type f -iname '*.mp3' -exec rename '/ /_/'  {} \;
text_cleaned: thi find command substitut space with underscor name replac space mp with
cmd_cleaned: find Path -type f -iname Regex -exec rename Regex {} \;
cmd_predicted: cptpt{ commsTYngfin wpan ciceyCheckingskipgeouticct

invocation: show the sum of disk used by all the files that belong to the user "test1" in the entire file system
cmd: find / -user test1 -exec du -sm {} \;|awk '{s+=$1}END{print s}'
text_cleaned: show sum of disk use by that belong to user test entir system
cmd_cleaned: find Path -user Regex -exec du -s -m {} \; | awk Program
cmd_predicted: cptpt{ commsTYngfin wpan ciceyCheckingskipgeoutilepan

invocation: test if the 2nd column in file a is in the reverse order of the second column of file b
cmd: diff --brief <(awk '{print $2}' A) <(tac B | awk '{print $2}')
text_cleaned: test if

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [39]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=3, min_delta=0.01):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [40]:
import os
from datetime import datetime
import shutil
from collections import defaultdict

import torch
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm, trange

#from apex import amp

class Trainer:

    def __init__(
            self, 
            model, 
            criterion,
            optimizer, 
            pad_token_id,
            device,
            stopper,
            logdir=None,
            max_grad_norm=-1
    ):
        """
            model: объект класса BertModel
            optimizer: оптимизатор
            pad_token_id: индекс паддинга. Нужен для создания attention mask
            device: девайс (cpu или cuda), на котором надо производить вычисления
            logdir: директория для записи логов
            max_grad_norm: максимум нормы градиентов, для клиппинга
        """
        self._criterion = criterion
        self._optimizer = optimizer
        self._pad_token_id = pad_token_id
        self._device = device
        self._logdir = logdir
        self._stopper = stopper
        self._max_grad_norm = max_grad_norm
        
        self._model = model.to(self._device)
        
        if self._logdir is not None:
            self._writer = SummaryWriter(log_dir=f"{logdir}/{datetime.now()}/", flush_secs=1)

        self._n_epoch = 0
        self._n_iter = 0

    def train(self, dataloaders, n_epochs):
        for epoch in trange(n_epochs):
            train_loss = self._train_step(dataloaders["train"])
            val_loss = self._val_step(dataloaders["valid"])
            self._n_epoch += 1
            print(f"Epoch: {self._n_epoch} | train_loss: {train_loss:.3f} | val_loss: {val_loss:.3f}")
            
            if self._logdir is not None:
                self._writer.add_scalar("loss/train", train_loss, global_step=self._n_epoch)
                self._writer.add_scalar("loss/val", val_loss, global_step=self._n_epoch)
                
            self._stopper(val_loss)
            if self._stopper.early_stop:
                break

    def _train_step(self, dataloader):
        """
            dataloader: объект класса DataLoader для обучения
        """
        self._model.train()
        epoch_loss = 0
        for text_idxs, cmd_idxs in dataloader:
            
            text_idxs = text_idxs.to(self._device)
            decoder_input = cmd_idxs[..., :-1].to(self._device)
            target = cmd_idxs[..., 1:].to(self._device)
            
            text_idxs_mask = torch.where(text_idxs != self._pad_token_id, 1, 0).to(self._device)
            decoder_input_mask = torch.where(decoder_input != self._pad_token_id, 1, 0).to(self._device)
            
            self._optimizer.zero_grad()        
            
            out = self._model(
                input_ids=text_idxs, decoder_input_ids=decoder_input,
                attention_mask=text_idxs_mask, decoder_attention_mask=decoder_input_mask)

            logits = out.logits

            loss = self._criterion(logits.reshape(-1, cmd_tokenizer.vocab_size()), target.reshape(-1))
            epoch_loss += loss.item()
            
            loss.backward()
            
            if self._max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(self._model.parameters(), self._max_grad_norm)
                
            self._optimizer.step()
        return epoch_loss / len(dataloader)
    
    def _val_step(self, dataloader):
        """
            dataloader: объект класса DataLoader для обучения
        """
        self._model.eval()
        epoch_loss = 0
        for text_idxs, cmd_idxs in dataloader:
            
            text_idxs = text_idxs.to(self._device)
            decoder_input = cmd_idxs[..., :-1].to(self._device)
            target = cmd_idxs[..., 1:].to(self._device)
            
            text_idxs_mask = torch.where(text_idxs != self._pad_token_id, 1, 0).to(self._device)
            decoder_input_mask = torch.where(decoder_input != self._pad_token_id, 1, 0).to(self._device)
            
            with torch.no_grad():
                out = self._model(
                    input_ids=text_idxs, decoder_input_ids=decoder_input,
                    attention_mask=text_idxs_mask, decoder_attention_mask=decoder_input_mask)
                logits = out.logits
                loss = self._criterion(logits.reshape(-1, cmd_tokenizer.vocab_size()), target.reshape(-1))
            
            epoch_loss += loss.item()
            
        return epoch_loss / len(dataloader)

In [41]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)
optimizer = torch.optim.Adam(model.parameters())
stopper = EarlyStopping(patience=2, min_delta=0.01)

In [42]:
device = torch.device('cuda')

In [43]:
trainer = Trainer(
    model=model, 
    criterion=criterion,
    optimizer=optimizer, 
    pad_token_id=PAD_ID,
    device=device,
    logdir="./runs",
    stopper=stopper,
    max_grad_norm=-1
)

In [44]:
trainer.train(loaders, 20)

  5%|▌         | 1/20 [00:13<04:16, 13.51s/it]

Epoch: 1 | train_loss: 1.603 | val_loss: 1.237


 10%|█         | 2/20 [00:26<04:01, 13.42s/it]

Epoch: 2 | train_loss: 0.784 | val_loss: 0.977


 15%|█▌        | 3/20 [00:40<03:48, 13.45s/it]

...

Epoch: 6 | train_loss: 0.434 | val_loss: 0.828


 35%|███▌      | 7/20 [01:33<02:53, 13.31s/it]

Epoch: 7 | train_loss: 0.397 | val_loss: 0.845
INFO: Early stopping counter 1 of 2


 35%|███▌      | 7/20 [01:46<03:17, 15.23s/it]

Epoch: 8 | train_loss: 0.364 | val_loss: 0.835
INFO: Early stopping counter 2 of 2
INFO: Early stopping


In [45]:
device = torch.device('cpu')

Результаты обучения на тренировочной выборке:

In [46]:
random_eval(train_data, train_ds, model)

invocation: display a long listing of the oldest file under '/hom/backups' directory tree
cmd: find /home/backups -printf "%T@ %p\n" | sort -n | head -1 | cut -d" " -f2- | xargs ls -al
text_cleaned: display a long list of oldest under hom backup tree
cmd_cleaned: find Path -printf "%T@ %p\n" | sort -n | head - Quantity | cut -d Regex -f Number | xargs -I {} ls -a -l {}
cmd_predicted: find Path -printf "%T@ %p\n" | sort

invocation: calculate the md5 sum of "password"
cmd: echo "password" | md5sum
text_cleaned: calcul md sum of password
cmd_cleaned: echo Regex | md5sum
cmd_predicted: md5sum File

invocation: run ls command on *.pl files
cmd: find . -name "*.pl" -exec ls -ld {} \;
text_cleaned: run ls command on pl
cmd_cleaned: find Path -name Regex -exec ls -l -d {} \;
cmd_predicted: ls -l | xargs -I {} ls -l -d {}

invocation: display all the files in the home folder which begin with "arrow"
cmd: find ~ -name 'arrow*'
text_cleaned: display home folder which begin with arrow
cmd_cleaned

Результаты на тестовой:

In [47]:
random_eval(valid_data, valid_ds, model)

invocation: ssh into "ssh.myhost.net" as user "myusername" and run command "mkdir -p $2"
cmd: ssh myusername@ssh.myhost.net "mkdir -p $2"
text_cleaned: ssh into ssh myhost net as user myusernam run command mkdir p
cmd_cleaned: ssh Regex command
cmd_predicted: ssh Regex command

invocation: split listing of the current directory into pieces per 500 lines named "outputxyznnn"
cmd: ls | split -l 500 - outputXYZ.
text_cleaned: split list of into piec per line name outputxyznnn
cmd_cleaned: ls | split -l Quantity File Regex
cmd_predicted: split File Regex

invocation: tar all the regular java files to myfile.tar
cmd: find . -type f -name "*.java" | xargs tar cvf myfile.tar
text_cleaned: tar regular java to myfil tar
cmd_cleaned: find Path -type f -name Regex | xargs -I {} tar -c -v -f File {}
cmd_predicted: find Path -type f -name Regex -exec tar -r -

invocation: split processed content of the file inout_file into pieces per 2000000 named as "out-prefix-nnn"
cmd: sed 's/\(.....\)\(.....\)/

## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей. 

In [ ]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1,
            device='cuda',
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id
        
        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature
        
        self.device = device
        
    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        ## YOUR CODE HERE ##

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [ ]:
beam_search_enginge = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=5,
    temperature=1, device='cuda',
)

In [ ]:
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        src = valid_ds.src[i]
        pred = beam_search_enginge.get_result(model, src)
        
        scores = []
        for x, proba in pred:
            pred_cmd = cmd_tokenizer.decode(list(map(int, x)))
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        print(max(scores))

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [ ]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []

    for i, (text, target_cmd) in enumerate(zip(df.text_cleaned.values, df.cmd.values)):
        input_tokens = ## YOUR CODE HERE ##
        predictions = beam_search_enginge.get_result(model, input_tokens)
        
        # get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(list(map(int, output_tokens)))
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)
        
        all_scores.append(max(object_scores))
    return all_scores

Если вы всё реализовали правильно, подобрали параметры BeamSearch то ваш средний скор на валидации должен быть >= 0.25, а скор на `handcrafted` части теста >= 0.13. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

In [ ]:
## YOUR CODE HERE ##

## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на `mined` >= 0 при скоре на `handrafted` >= 0.16.

In [ ]:
## YOUR CODE HERE ##

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...